In [162]:
import pandas as pd
import re
import os

# ---- Учитај фајлове ----
df1 = pd.read_csv("IMSBS-2000_heuristic-h5_imbs-iters-500_num-roots-10.csv")  
df2 = pd.read_csv("LIMSBS-2000_imbs-iters-500_num-roots-10_U5_5_5_A2_F3.csv") #"learning_U10_5_5_A2_F3.csv")   #LIMSBS-1000_imbs-iters-500_num-roots-10_U5_5_5_A1_F3.csv
# TODO (executed experiments): LIMSBS-1000_imbs-iters-500_num-roots-10 U10 5 5 A2 F3

# ---- Извуци 4 броја из имена фајла ----
def extract_numbers(filename):
    # узимамо сва 4 броја из имена: n_m_k_index
    m = re.match(r".*?_(\d+)_(\d+)_(\d+)_(\d+)\.txt\.out", filename)
    if m:
        return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4))
    else:
        return None, None, None, None

for df in [df1, df2]:
    df[['n','m','k','idx']] = df['file'].apply(lambda x: pd.Series(extract_numbers(x)))
    # kreiramo key po prva tri broja
    df['key'] = df[['n','m','k']].astype(str).agg('_'.join, axis=1)

# ---- Spoji podatke po key ----
merged = pd.merge(df1, df2, on='key', suffixes=('_1','_2'))

# ---- Agregiraj po prva tri broja (key) ----
agg = merged.groupby('key').agg({
    'time_1': 'mean',
    'quality_1': 'mean',
    'time_2': 'mean',
    'quality_2': 'mean'
}).reset_index()

# ---- Sačuvaj u CSV ----
#agg.to_csv("merged_aggregated.csv", index=False)

# ---- Rezultat ----
agg = agg.rename(columns={"quality_1" : "IMSBS-2000_heuristic-h5_imbs-iters-500_num-roots-10", "time_1": "IMSBS-time", "quality_2": "LIMSBS-2000_imbs-iters-500_num-roots-10_U10_5_5_A2_F3", "time_2": "LIMSBS-time"})


In [165]:
agg[["key", "IMSBS-2000_heuristic-h5_imbs-iters-500_num-roots-10", "LIMSBS-2000_imbs-iters-500_num-roots-10_U10_5_5_A2_F3", "IMSBS-time", "LIMSBS-time" ]].mean() #.to_latex("results_preliminary.tex", index=False)

key                                                      3.156316e+157
IMSBS-2000_heuristic-h5_imbs-iters-500_num-roots-10       6.468438e+01
LIMSBS-2000_imbs-iters-500_num-roots-10_U10_5_5_A2_F3     6.506563e+01
IMSBS-time                                                1.015726e+01
LIMSBS-time                                               2.813454e+01
dtype: float64

In [166]:
#64.596875 (LIMSBS) vs. 63.668750 (IMSBS)